In [ ]:
from google.cloud import bigquery
import pandas as pd
from pprint import pprint
import json
import numpy as np

In [ ]:
 client = bigquery.Client(location="US")

In [3]:
query = f""" SELECT
 job_id,
 creation_time,
 query,
 state
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE date(creation_time) >= current_date and state != "DONE" """

#df = client.query(query).to_dataframe()
#print(df)

In [4]:
def get_ltv(retailer, pii_type,  date_start, date_end):
    
    query = f"""
    with sub_df as(
    select pii_type, cdp_pii_hash,event_ts from `narvar-research.iris_commons.consumer_event_orders_unpivot_merged_deduped` 
    where retailer = '{retailer}' and pii_type = '{pii_type}' and 
    event_ts >= '{date_start}' and event_ts <= '{date_end}' group by 1,2,3
    ),
    sub_df2 as
    (
    select pii_type, cdp_pii_hash, event_ts from `narvar-research.iris_commons.consumer_event_orders_unpivot_merged_deduped`
    where retailer = '{retailer}' and pii_type = '{pii_type}' and event_ts > '{date_start}'
    group by 1,2,3
    )
    select a.pii_type, a.cdp_pii_hash, b.event_ts as next_order_date,
    DATE_DIFF(date(b.event_ts),date(a.event_ts),DAY) AS next_order_diff 
    from sub_df a 
    left join
    sub_df2 b 
        on a.pii_type = b.pii_type and a.cdp_pii_hash = b.cdp_pii_hash and a.event_ts < b.event_ts
        QUALIFY ROW_NUMBER() OVER(PARTITION BY a.pii_type, a.cdp_pii_hash ORDER BY b.event_ts ASC) = 1
    """
    
    next_df = client.query(query).to_dataframe()
    
    query = f"""
    with sub_df as(
    select pii_type, cdp_pii_hash from `narvar-research.iris_commons.consumer_event_orders_unpivot_merged_deduped` 
    where retailer = '{retailer}' and pii_type = '{pii_type}' and 
    event_ts >= '{date_start}' and event_ts <= '{date_end}'
    )
    select a.pii_type, a.cdp_pii_hash,
    DATE_DIFF('{date_start}',DATE(min(b.event_ts)),DAY) AS prev_order_diff 
    from sub_df a 
    left join
    `narvar-research.iris_commons.consumer_event_orders_unpivot_merged_deduped` b 
        on a.pii_type = b.pii_type and a.cdp_pii_hash = b.cdp_pii_hash
    where b.retailer = '{retailer}' and b.pii_type = '{pii_type}' and b.event_ts < '{date_start}'
    group by 1,2
    """
    
#    prev_df = client.query(query).to_dataframe()
    
    for days in (90, 180, 270, 365):
        next_df[f"LTV_in_next_{days}"] = (next_df["next_order_diff"] <= days).astype(float)
#        prev_df[f"LTV_in_prev_{days}"] = (prev_df["prev_order_diff"] <= days).astype(float)
    
#    final_df = pd.merge(next_df, prev_df, on = ["pii_type", "cdp_pii_hash"], how = "left")
    #final_df = final_df.fillna(0)
    return next_df

In [5]:
final_df = get_ltv("puma", "EMAIL",  "2023-06-01", "2023-06-30")

In [6]:
final_df.head()

,pii_type,cdp_pii_hash,next_order_date,next_order_diff,LTV_in_next_90,LTV_in_next_180,LTV_in_next_270,LTV_in_next_365
0,EMAIL,-I2J3TEjqobKFYnuyK1aMIGa7vecRoSWCDBEqJRZJRw,2023-07-08 02:37:58+00:00,10,1.0,1.0,1.0,1.0
1,EMAIL,-KLChQ1v2P0k6elbbaAJgJiGHETxmN2gkDExewpw63o,2024-01-29 20:12:47+00:00,234,0.0,0.0,1.0,1.0
2,EMAIL,-NnvohoZbcXxh260JI_RTF5yXKXzkaOaSLppBNsso2k,2023-06-09 14:54:40+00:00,3,1.0,1.0,1.0,1.0
3,EMAIL,-ghdRKnrD-JTFdAx1e47DMZv-Ory2G33aoMaGcZcCus,2023-12-11 23:01:56+00:00,182,0.0,0.0,1.0,1.0
4,EMAIL,-sKlc15m5X1-BYQTvWLzpzLsp3b1bWx59DDhgTr2X8o,2024-01-04 03:13:04+00:00,204,0.0,0.0,1.0,1.0


In [6]:
%%time
retailers = ["levi", "yankeecandle", "7forallmankind", "johnnywas",
            "containerstore", "smithoptics", "drmartens",
            "petermillar", "puma", "hibbett", "bostonproper",
            "off5th","brooksbrothers", "aeropostale"]
retailers = ["petermillar"]
pii = "EMAIL"
result_df = []
for retailer_ in retailers:
    print(retailer_)
    for m_,s_,e_ in [("Jan_2023", "2023-01-01", "2023-01-31"),
                 ("Feb_2023", "2023-02-01", "2023-02-28"),
                 ("Mar_2023", "2023-03-01", "2023-03-31"),
                 ("Apr_2023", "2023-04-01", "2023-04-30"),
                 ("May_2023", "2023-05-01", "2023-05-31"),
                 ("Jun_2023", "2023-06-01", "2023-06-30"),]:
        print(m_)   
        final_df = get_ltv(retailer_, "EMAIL",  s_, e_)
        temp_df= (final_df[["LTV_in_next_90","LTV_in_next_180", "LTV_in_next_270","LTV_in_next_365"]].sum( axis = 0) / len(final_df)).reset_index()
        temp_df.columns = ["LTV Index", f"LTV_%_FOR_PII_IN_MONTH_{m_.upper()}"]
        temp_df["retailer"] = retailer_
        result_df.append(temp_df)

result_df = pd.concat(result_df,axis = 1)
result_df = result_df.loc[:, ~result_df.columns.duplicated()]
result_df        

petermillar
Jan_2023
Feb_2023
Mar_2023
Apr_2023
May_2023
Jun_2023
CPU times: user 421 ms, sys: 109 ms, total: 530 ms
Wall time: 51.1 s


,LTV Index,LTV_%_FOR_PII_IN_MONTH_JAN_2023,retailer,LTV_%_FOR_PII_IN_MONTH_FEB_2023,LTV_%_FOR_PII_IN_MONTH_MAR_2023,LTV_%_FOR_PII_IN_MONTH_APR_2023,LTV_%_FOR_PII_IN_MONTH_MAY_2023,LTV_%_FOR_PII_IN_MONTH_JUN_2023
0,LTV_in_next_90,0.234664,petermillar,0.274015,0.281667,0.288433,0.303974,0.269753
1,LTV_in_next_180,0.346818,petermillar,0.405780,0.402246,0.391012,0.372375,0.349442
2,LTV_in_next_270,0.432658,petermillar,0.454651,0.464870,0.456958,0.446827,0.409328
3,LTV_in_next_365,0.498503,petermillar,0.522574,0.515822,0.500105,0.484220,0.448102
